In [2]:
import re
import json
import requests

from xml.etree.ElementTree import tostring

In [3]:
def parse_answer(line, p):
    obj = re.findall(p, line)
    if not obj:
        return None
    return obj[0].split("@")
     
def parse_comment(line, p):
    obj = re.findall(p, line)
    if not obj:
        return None
    lists = obj[0].split("#")
    for l in lists:
        if l.startswith('"'):
            l = json.loads(l)

    return lists

In [4]:

def md_qa_parse(real_link):

    qaparts = {}
    response = ''
    quiz_count = 0
    answers = []
    comments = []

    block_pattern = re.compile('{%(\w*|[^%{}@]*@[^%]*)%}', re.M)
    inline_pattern = re.compile('{%(\w*|[^%{}@]*@[^%]*)%}')
    r = requests.get(real_link)

    if not r.ok:
        return {'response': False, 'info': real_link+u' not exists'}

    if 'content-length' in r.headers and \
        int(r.headers['content-length']) > 8 * 5000 * 1024 * 3:
        return {'response': False, 'info': real_link+u' 太长'}
   
    line_count = 0
    block_flag = False
    
    for line in r.iter_lines():
        
        line_count += 1
        
        if block_flag:
            
            temp1 = parse_answer(line, '@([^#]*)')
            temp2 = parse_comment(line, '#([^%]*)')
            if temp1:
                answer.append(temp1)
                continue
            elif temp2:
                comments.append(temp2)
            elif line.find("%}") >= 0:
                if len(answer) == 1:
                    answer = answer[0]
                if len(comment) == 1:
                    comment = comment[0]
                answers.append(answer)
                comments.append(comment)
                block_flag = False
                response += line
            else:
                response += line
            continue
            
        if line_count > 5000:
            return {'response': False, 'info': real_link+u' 太长, 超过5000行'}
        
        if not line.startswith("{%"):
            response += line + '\n'
            continue
            
        lists = re.findall('{%(\w*|[^%{}@]*@[^%]*)%}', line)
        if lists:
            answer = parse_answer(line, '@([^#]*)')
            answers.append(answer)
            comment = parse_comment(line, '#([^%]*)')
            comments.append(comment)
            response += line[:line.find("@")] + '%}\n'
        else:
            answer = []
            comment = []
            if line.find("@") < 0:
                response += line + "\n"
            else:
                response += line[:line.find("@")] + "\n"
            block_flag = True
            
    qaparts['response'] = response
    qaparts['answer'] = answers
    qaparts['comment'] = comments
    
    return qaparts

In [5]:
z = md_qa_parse("http://localhost:4321/%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD/%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0/%E7%BA%BF%E6%80%A7%E6%A8%A1%E5%9E%8B/%E9%80%BB%E8%BE%91%E5%9B%9E%E5%BD%92/%E9%80%BB%E8%BE%91%E5%9B%9E%E5%BD%92%E5%9F%BA%E7%A1%80.md")

In [ ]:
print z['comment']
print 'comment'
for e in  z['comment']:
    print e[0]
    #print isinstance(e[0], str)

In [ ]:
print len(z['answer'])
for e in  z['answer']:
    print e
    print isinstance(e[0], str)
    


In [9]:
print  len(z['response'])

5795


In [ ]:
print z['response']

In [23]:
x = "{%asdf}"
print x.startswith("{%")

True
